In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

import re
import string

import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('stopwords')

In [2]:
def load_stop_words(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stop_words = set(file.read().splitlines())
    return stop_words

In [3]:
dataset = pd.read_json("dataset\Turkish_data_depository_630koseyazisi.jsonl",lines=True)
print(dataset.head(10))
print(dataset.info())

  writer  file_id                                             column
0    ttu       27  O zaman 28 Şubat neden yapıldı?  \r\n     \r\n...
1    ttu       28  O zaman 28 Şubat neden yapıldı?  \r\n     \r\n...
2    ttu        3  Cumhuriyetin en kritik yerel seçimi \r\nMURAT ...
3    ttu        4  Çekilin artık kamera karşısından \r\nHER kar y...
4    ttu       23  Lorant'ın heyecan veren hedefi  \r\n     \r\n\...
5    ttu       14  Gerçekten de gülüp geçilecek bir öneri \r\nGAZ...
6    ttu       13  Mercedes çeşitlemeleri \r\nTÜRK insanı, Merced...
7    ttu       31  Yazarın çilesi  \r\n     \r\n\r\n  \r\nZAMAN z...
8    ttu       19  Tüm yazı konularını silip süpüren fotoğraf \r\...
9    ttu       11  Şarkta böyledir bu işler...  \r\n     \r\nttu...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   writer   630 non-null    object
 1   file_id  630 non-null

In [14]:
x = dataset.iloc[:, 2].values.astype("str") # author text
y = dataset.iloc[:, 0].values.astype("str") # author name

print(f"type x : {type(x)} , type y : {type(y)}")

type x : <class 'numpy.ndarray'> , type y : <class 'numpy.ndarray'>


In [28]:
stop_words =load_stop_words("stopwordsTR\stopwords.txt") #set(stopwords.words('turkish'))

from TurkishStemmer import TurkishStemmer

# Lemmatizasyon ve stemming işlemleri için hazırlık
stemmer = TurkishStemmer()

# Her bir metni lemmatizasyon ve stemming işlemlerinden geçirme
for i in range(len(x)):
    # Convert to lowercase - Küçük harfe çevirme
    x[i] = x[i].lower()
    
    # Remove escape characters - Kaçış karakterlerini kaldırma
    x[i] = re.sub(r'[\r\n]', ' ', x[i]) 
    
    # Remove unnecessary characters - Gereksiz karakterleri kaldırma
    x[i] = re.sub(r'[^a-zA-ZğüşıöçĞÜŞİÖÇ\s]', '', x[i])
    
    # Tokenization - Kelime ayırma
    tokens = word_tokenize(x[i])
    
    # Remove stop words - Stop kelimeleri kaldırma
    filtered_tokens = [word for word in tokens if word not in stop_words]
    x[i] = ' '.join(filtered_tokens)
    
    # Stemming
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    x[i] = ' '.join(stemmed_tokens)
    
    # Remove punctuation marks - Noktalama işaretlerini kaldırma
    x[i] = x[i].translate(str.maketrans('', '', string.punctuation))
    
    # Remove repeating spaces - Tekrar eden boşlukları kaldırma
    x[i] = re.sub(r'\s+', ' ', x[i])

In [29]:
# X verisini unigram TF-IDF vektörlerine dönüştürme
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X_vec = vectorizer.fit_transform(x)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

In [31]:
from sklearn.model_selection import GridSearchCV, cross_val_score

# SGDClassifier için hiperparametrelerin belirlenmesi
param_grid = {
    'alpha': [1e-05, 0.0001, 0.001],
    'max_iter': [50, 100, 200],
    'penalty': ['elasticnet', 'l2']
}

In [27]:
# GridSearchCV ile model oluşturma
grid_search = GridSearchCV(SGDClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# En iyi parametreleri ve skorları gösterme
print("En iyi parametreler:", grid_search.best_params_)
print("En iyi skor:", grid_search.best_score_)

# Çapraz doğrulama ile model performansını değerlendirme
cross_val_scores = cross_val_score(grid_search, X_test, y_test, cv=5)
print("Çapraz doğrulama skorları:", cross_val_scores)
print("Ortalama çapraz doğrulama skoru:", cross_val_scores.mean())

En iyi parametreler: {'alpha': 0.001, 'max_iter': 50, 'penalty': 'l2'}
En iyi skor: 0.7816633663366337


c:\Users\Umitk\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Umitk\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Umitk\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Umitk\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Umitk\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:73

Çapraz doğrulama skorları: [0.46153846 0.6        0.64       0.72       0.44      ]
Ortalama çapraz doğrulama skoru: 0.5723076923076923


In [32]:
# SGDClassifier ile model oluşturma
model = SGDClassifier(alpha=1e-05, max_iter=50, penalty='elasticnet', random_state=42)

# Modeli eğitme
model.fit(X_train, y_train)

# Modelin performansını değerlendirme
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy:.4f}')

Test accuracy: 0.7063
